# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [1]:
import graphlab

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [2]:
def polynomial_sframe(feature, degree):
        # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name]=feature.apply(lambda x: x**power)

    return poly_sframe
    

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Awan\AppData\Local\Temp\graphlab_server_1479641892.log.0


This non-commercial license of GraphLab Create for academic use is assigned to mmohsin3691@gmail.com and will expire on October 12, 2017.


As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [5]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [6]:
l2_small_penalty = 1e-5

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [9]:
poly1_data = polynomial_sframe(sales['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features = poly1_data.column_names() 
poly1_data['price'] = sales['price']

model1 = graphlab.linear_regression.create(poly1_data, target = 'price', features = my_features, validation_set = None, l2_penalty=l2_small_penalty)
model1.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.031712     | 2662555.737393     | 245656.462164 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 16

Data:
+-------------+-------+--------------------+--------+
|     name    | index |       value        | stderr |
+-------------+-------+--------------------+--------+
| (intercept) |  None |   167924.863088    |  nan   |
|   power_1   |  None |   103.090934871    |  nan   |
|   power_2   |  None |   0.13460457027    |  nan   |
|   power_3   |  None | -0.000129071375318 |  nan   |
|   power_4   |  None | 5.18928995405e-08  |  nan   |
|   power_5   |  None |  -7.771693847e-12  |  nan   |
|   power_6   |  None | 1.71144973107e-16  |  nan   |
|   power_7   |  None | 4.51177797327e-20  |  nan   |
|   power_8   |  None | -4.78838493488e-25 |  nan   |
|   power_9   |  None | -2.33343501458e-28 |  nan   |
+-------------+-------+--------------------+--------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [10]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [14]:
set1_data = polynomial_sframe(set_1['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features_1 = set1_data.column_names() 
set1_data['price'] = set_1['price']
model1 = graphlab.linear_regression.create(set1_data, target = 'price', features = my_features_1, validation_set = None, l2_penalty=l2_small_penalty)
model1.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5404

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.019051     | 2191984.901477     | 248699.117253 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 16

Data:
+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |    9306.4666294    |   810199.413847   |
|   power_1   |  None |   585.865804883    |   3442.10422346   |
|   power_2   |  None |  -0.397305874315   |   6.02291333786   |
|   power_3   |  None | 0.000141470888681  |  0.0057279761005  |
|   power_4   |  None | -1.52945955404e-08 | 3.29578977423e-06 |
|   power_5   |  None | -3.79756819869e-13 |  1.2133445418e-09 |
|   power_6   |  None | 5.97481991752e-17  | 3.00417024061e-13 |
|   power_7   |  None | 1.06888523817e-20  | 5.50094917614e-17 |
|   power_8   |  None | 1.59343780943e-25  | 8.36424155782e-21 |
|   power_9   |  None | -6.92834881362e-29 | 1.00031911962e-24 |
+-------------+-------+--------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [16]:
set2_data = polynomial_sframe(set_2['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features_2 = set2_data.column_names() 
set2_data['price'] = set_2['price']
model2 = graphlab.linear_regression.create(set2_data, target = 'price', features = my_features_2, validation_set = None, l2_penalty=l2_small_penalty)
model2.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5398

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.018052     | 1975178.190939     | 234533.610645 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 16

Data:
+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   -25115.8993262   |    1461908.2824   |
|   power_1   |  None |   783.493780749    |   8141.54862382   |
|   power_2   |  None |  -0.767759273211   |   19.1599476904   |
|   power_3   |  None | 0.000438766345759  |  0.0252348495278  |
|   power_4   |  None | -1.15169156529e-07 | 2.07713736269e-05 |
|   power_5   |  None | 6.84281125926e-12  |  1.1267705289e-08 |
|   power_6   |  None | 2.51195199365e-15  | 4.11595289605e-12 |
|   power_7   |  None | -2.0644054241e-19  | 1.00652509326e-15 |
|   power_8   |  None | -4.59673169746e-23 | 1.56967454249e-19 |
|   power_9   |  None | -2.71272039208e-29 | 1.26503325439e-23 |
+-------------+-------+--------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [17]:

set3_data = polynomial_sframe(set_3['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features_3 = set3_data.column_names() 
set3_data['price'] = set_3['price']
model3 = graphlab.linear_regression.create(set3_data, target = 'price', features = my_features_3, validation_set = None, l2_penalty=l2_small_penalty)
model3.get("coefficients")



Linear regression:

--------------------------------------------------------

Number of examples          : 5409

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.016042     | 2283722.683416     | 251097.728068 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 16

Data:
+-------------+-------+--------------------+-------------------+
|     name    | index |       value        |       stderr      |
+-------------+-------+--------------------+-------------------+
| (intercept) |  None |   462426.567754    |   1264112.98769   |
|   power_1   |  None |   -759.251854206   |   6160.31746987   |
|   power_2   |  None |   1.02867006808    |   12.6575879778   |
|   power_3   |  None | -0.00052826454477  |  0.0145109950169  |
|   power_4   |  None | 1.15422915761e-07  | 1.03360324025e-05 |
|   power_5   |  None | -2.26096108253e-12 | 4.79760227635e-09 |
|   power_6   |  None | -2.08214269714e-15 | 1.46673002389e-12 |
|   power_7   |  None | 4.08770281089e-20  | 2.86235422373e-16 |
|   power_8   |  None | 2.57079169816e-23  | 3.16708303132e-20 |
|   power_9   |  None |  1.2431125129e-27  | 2.07861266245e-24 |
+-------------+-------+--------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [18]:
set4_data = polynomial_sframe(set_4['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features_4 = set4_data.column_names() 
set4_data['price'] = set_4['price']
model4 = graphlab.linear_regression.create(set4_data, target = 'price', features = my_features_4, validation_set = None, l2_penalty=l2_small_penalty)
model4.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5402

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.018050     | 2378292.371871     | 244341.293202 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 16

Data:
+-------------+-------+--------------------+---------------+
|     name    | index |       value        |     stderr    |
+-------------+-------+--------------------+---------------+
| (intercept) |  None |   -170240.034765   | 1038851.01649 |
|   power_1   |  None |   1247.59034807    | 5068.70200366 |
|   power_2   |  None |   -1.22460911823   | 5.25072710262 |
|   power_3   |  None | 0.000555254617758  |      nan      |
|   power_4   |  None | -6.38262314137e-08 |      nan      |
|   power_5   |  None | -2.20216008557e-11 |      nan      |
|   power_6   |  None |  4.8183470755e-15  |      nan      |
|   power_7   |  None | 4.21461639711e-19  |      nan      |
|   power_8   |  None | -7.99880731487e-23 |      nan      |
|   power_9   |  None | -1.32365918453e-26 |      nan      |
+-------------+-------+--------------------+---------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [19]:
poly1_data = polynomial_sframe(sales['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features = poly1_data.column_names() 
poly1_data['price'] = sales['price']

model1 = graphlab.linear_regression.create(poly1_data, target = 'price', features = my_features, validation_set = None, l2_penalty=1e5)
model1.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.024064     | 4399802.043621     | 322802.393717 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 16

Data:
+-------------+-------+--------------------+--------+
|     name    | index |       value        | stderr |
+-------------+-------+--------------------+--------+
| (intercept) |  None |   487313.786085    |  nan   |
|   power_1   |  None |   7.63337888205    |  nan   |
|   power_2   |  None |  0.0040966574976   |  nan   |
|   power_3   |  None |  7.7176014244e-07  |  nan   |
|   power_4   |  None |  5.5885134377e-11  |  nan   |
|   power_5   |  None |  2.5213782703e-15  |  nan   |
|   power_6   |  None | 9.62987628519e-20  |  nan   |
|   power_7   |  None |  3.0505784985e-24  |  nan   |
|   power_8   |  None |  3.8087812483e-29  |  nan   |
|   power_9   |  None | -6.37982650955e-33 |  nan   |
+-------------+-------+--------------------+--------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [20]:
set2_data = polynomial_sframe(set_2['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features_2 = set2_data.column_names() 
set2_data['price'] = set_2['price']
model2 = graphlab.linear_regression.create(set2_data, target = 'price', features = my_features_2, validation_set = None, l2_penalty=1e5)
model2.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5398

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.017050     | 2984894.541944     | 323238.809634 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 16

Data:
+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   519216.897383   |    2014830.5895   |
|   power_1   |  None |   2.04470474182   |   11220.8415608   |
|   power_2   |  None |  0.0011314362684  |   26.4066146726   |
|   power_3   |  None | 2.93074277549e-07 |   0.034779163209  |
|   power_4   |  None | 4.43540598453e-11 | 2.86275134176e-05 |
|   power_5   |  None | 4.80849112204e-15 |  1.5529371824e-08 |
|   power_6   |  None | 4.53091707826e-19 | 5.67268678875e-12 |
|   power_7   |  None | 4.16042910575e-23 | 1.38721257101e-15 |
|   power_8   |  None | 3.90094635128e-27 | 2.16335615706e-19 |
|   power_9   |  None |  3.7773187602e-31 | 1.74349357506e-23 |
+-------------+-------+-------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [21]:

set3_data = polynomial_sframe(set_3['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features_3 = set3_data.column_names() 
set3_data['price'] = set_3['price']
model3 = graphlab.linear_regression.create(set3_data, target = 'price', features = my_features_3, validation_set = None, l2_penalty=1e5)
model3.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5409

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.019051     | 3695342.767093     | 350033.521294 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 16

Data:
+-------------+-------+-------------------+-------------------+
|     name    | index |       value       |       stderr      |
+-------------+-------+-------------------+-------------------+
| (intercept) |  None |   522911.518048   |   1762190.05962   |
|   power_1   |  None |   2.26890421877   |    8587.5632283   |
|   power_2   |  None |  0.00125905041842 |   17.6448434043   |
|   power_3   |  None | 2.77552918155e-07 |  0.0202285170891  |
|   power_4   |  None |  3.2093309779e-11 | 1.44085645294e-05 |
|   power_5   |  None | 2.87573572364e-15 | 6.68792040245e-09 |
|   power_6   |  None | 2.50076112671e-19 | 2.04464086154e-12 |
|   power_7   |  None | 2.24685265906e-23 | 3.99015927317e-16 |
|   power_8   |  None | 2.09349983135e-27 | 4.41495522166e-20 |
|   power_9   |  None | 2.00435383296e-31 | 2.89761327289e-24 |
+-------------+-------+-------------------+-------------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [22]:
set4_data = polynomial_sframe(set_4['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features_4 = set4_data.column_names() 
set4_data['price'] = set_4['price']
model4 = graphlab.linear_regression.create(set4_data, target = 'price', features = my_features_4, validation_set = None, l2_penalty=1e5)
model4.get("coefficients")

Linear regression:

--------------------------------------------------------

Number of examples          : 5402

Number of features          : 15

Number of unpacked features : 15

...

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.016045     | 3601895.280124     | 323111.582889 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Columns:
	name	str
	index	str
	value	float
	stderr	float

Rows: 16

Data:
+-------------+-------+-------------------+---------------+
|     name    | index |       value       |     stderr    |
+-------------+-------+-------------------+---------------+
| (intercept) |  None |   513667.087087   | 1373753.86668 |
|   power_1   |  None |   1.91040938244   | 6702.74068754 |
|   power_2   |  None |  0.00110058029175 | 6.94344669789 |
|   power_3   |  None | 3.12753987879e-07 |      nan      |
|   power_4   |  None | 5.50067886825e-11 |      nan      |
|   power_5   |  None | 7.20467557825e-15 |      nan      |
|   power_6   |  None | 8.24977249384e-19 |      nan      |
|   power_7   |  None | 9.06503223498e-23 |      nan      |
|   power_8   |  None | 9.95683160453e-27 |      nan      |
|   power_9   |  None | 1.10838127982e-30 |      nan      |
+-------------+-------+-------------------+---------------+
[16 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [23]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [24]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print i, (start, end)

0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [25]:
train_valid_shuffled[0:10] # rows 0 to 9

Columns:
	id	str
	date	datetime
	price	float
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	int
	floors	str
	waterfront	int
	view	int
	condition	int
	grade	int
	sqft_above	int
	sqft_basement	int
	yr_built	int
	yr_renovated	int
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float

Rows: 10

Data:
+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 2780400035 | 2014-05-05 00:00:00+00:00 | 665000.0 |   4.0    |    2.5    |
| 1703050500 | 2015-03-21 00:00:00+00:00 | 645000.0 |   3.0    |    2.5    |
| 5700002325 | 2014-06-05 00:00:00+00:00 | 640000.0 |   3.0    |    1.75   |
| 0475000510 | 2014-11-18 00:00:00+00:00 | 594000.0 |   3.0    |    1.0    |
| 0844001052 | 2015-01-28 00:00:00+00:00 | 365000.0 |   4.0    |    2.5    |
| 2781280290 | 2015-04-27 00:00:00+00:00 | 305000.0 |   3.0    |    2.5    |
| 2214800630 | 2014-11-05 00:00:00+00:00 | 239950.0 |   3.0    |    2.25   |
| 2114700540 | 2014-10-21 00:00:00+00:00 | 366000.0 |   3.0    |    2.5    |
| 2596400050 | 2014-07-30 00:00:00+00:00 | 375000.0 |   3.0    |    1.0    |
| 4140900050 | 2015-01-26 00:00:00+00:00 | 440000.0 |   4.0    |    1.75   |
+------------+---------------------------+----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    2800.0   |   5900   |   1    |     0      |  0   |     3     |   8   |    1660    |
|    2490.0   |   5978   |   2    |     0      |  0   |     3     |   9   |    2490    |
|    2340.0   |   4206   |   1    |     0      |  0   |     5     |   7   |    1170    |
|    1320.0   |   5000   |   1    |     0      |  0   |     4     |   7   |    1090    |
|    1904.0   |   8200   |   2    |     0      |  0   |     5     |   7   |    1904    |
|    1610.0   |   3516   |   2    |     0      |  0   |     3     |   8   |    1610    |
|    1560.0   |   8280   |   2    |     0      |  0   |     4     |   7   |    1560    |
|    1320.0   |   4320   |   1    |     0      |  0   |     3     |   6   |    660     |
|    1960.0   |   7955   |   1    |     0      |  0   |     4     |   7   |    1260    |
|    2180.0   |  10200   |   1    |     0      |  2   |     3     |   8   |    2000    |
+-------------+----------+--------+------------+------+-----------+-------+------------+
+---------------+----------+--------------+---------+-------------+
| sqft_basement | yr_built | yr_renovated | zipcode |     lat     |
+---------------+----------+--------------+---------+-------------+
|      1140     |   1963   |      0       |  98115  | 47.68093246 |
|       0       |   2003   |      0       |  98074  | 47.62984888 |
|      1170     |   1917   |      0       |  98144  | 47.57587004 |
|      230      |   1920   |      0       |  98107  | 47.66737217 |
|       0       |   1999   |      0       |  98010  | 47.31068733 |
|       0       |   2006   |      0       |  98055  | 47.44911017 |
|       0       |   1979   |      0       |  98001  | 47.33933392 |
|      660      |   1918   |      0       |  98106  | 47.53271982 |
|      700      |   1963   |      0       |  98177  | 47.76407345 |
|      180      |   1966   |      0       |  98028  | 47.76382378 |
+---------------+----------+--------------+---------+-------------+
+---------------+---------------+-----+
|      long     | sqft_living15 | ... |
+---------------+---------------+-----+
| -122.28583258 |     2580.0    | ... |
| -122.02177564 |     2710.0    | ... |
|   -122.28796  |     1360.0    | ... |
| -122.36472902 |     1700.0    | ... |
|  -122.0012452 |     1560.0    | ... |
|  -122.1878086 |     1610.0    | ... |
| -122.25864364 |     1920.0  

Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [35]:
validation4 = train_valid_shuffled[5818:7758]

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [36]:
print int(round(validation4['price'].mean(), 0))

536234


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [ ]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [37]:
train4=train_valid_shuffled[0:5818].append(train_valid_shuffled[7758:19396])


To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [38]:
print int(round(train4['price'].mean(), 0))

539450


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [46]:
def get_RSS(prediction, output):
    residual = output - prediction
    # square the residuals and add them up
    RS = residual*residual
    RSS = RS.sum()
    return(RSS)

def k_fold_cross_validation(k, l2_penalty, data, features_list):
    n = len(data)
    RSS = 0
    for i in xrange(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        validation=data[start:end+1]
        train=data[0:start].append(data[end+1:n])
        model = graphlab.linear_regression.create(train, target='price', features = features_list, l2_penalty=l2_penalty,validation_set=None,verbose = False)
        predictions=model.predict(validation)
        A =get_RSS(predictions,validation['price'])
        RSS = RSS + A
        
    Val_err = RSS/k
    return Val_err

Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [47]:
poly_data = polynomial_sframe(train_valid_shuffled['sqft_living'], 15)

my_features = poly_data.column_names()

poly_data['price'] = train_valid_shuffled['price']

for l2_penalty in np.logspace(1, 7, num=13):

    Val_err = k_fold_cross_validation(10, l2_penalty, poly_data, my_features)
    print l2_penalty
    print Val_err

10.0
4.91826427769e+14
31.6227766017
2.87504229919e+14
100.0
1.60908965822e+14
316.227766017
1.22090967326e+14
1000.0
1.21192264451e+14
3162.27766017
1.2395000929e+14
10000.0
1.36837175248e+14
31622.7766017
1.71728094842e+14
100000.0
2.2936143126e+14
316227.766017
2.52940568729e+14
1000000.0
2.58682548441e+14
3162277.66017
2.62819399742e+14
10000000.0
2.64889015378e+14


***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [53]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.
poly1_data = polynomial_sframe(train_valid_shuffled['sqft_living'], 15) # use equivalent of `polynomial_sframe`
my_features = poly1_data.column_names() 
poly1_data['price'] = train_valid_shuffled['price']
model1 = graphlab.linear_regression.create(poly1_data, target = 'price', features = my_features, verbose = False, validation_set = None, l2_penalty=1000)
Val_err = k_fold_cross_validation(10, 1000, poly1_data, my_features)
print Val_err


1.21192264451e+14


Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`. This way, your final model will be trained on the entire dataset.

1.95479922778e+18


***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***

In [55]:
poly_test = polynomial_sframe(test['sqft_living'], 15)
poly_test['price'] = test['price']
print sum((model1.predict(poly_test[my_features]) - poly_test['price'])**2)


1.28780855058e+14
